In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [9]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}



In [10]:
def string_to_number(my_string, data_kind):
    my_number = '-'
    
    try:
        heap, sep, tail = my_string.partition(': ') 
        
        if(data_kind == 'costs'):
            heap, sep, tail = tail.partition(' ')
    
            if("Mio. €" in tail):
                my_number = float(heap.replace(',','.'))
            
            elif("Tsd. €" in tail):
                my_number = float(heap.replace(',','.'))/1000
             
        if(data_kind == 'age'):
            my_number = tail   
    
    except:
        print("falsche Werte")
        
    return my_number

In [11]:
club = "fc-bayern-munchen"
df_total = pd.DataFrame()
durchlauf = 0
current_season = 1960


while current_season <= 2018:
    
    # Auf diesen Link greift der Crawler zu -> Link zur Transferseite des geünschten Vereins
    
    page = "https://www.transfermarkt.tv/" + club + "/transfers/verein/27/plus/1?saison_id=" + str(current_season) + "&pos=&detailpos=&w_s="
    page_tree = requests.get(page, headers=headers)
    page_soup = BeautifulSoup(page_tree.content, 'html.parser')
    
    # Name des Vereins wird aus der Seite ausgelesen
    
    ele_name_club = page_soup.find_all("h1", {"itemprop": "name"})
    name_club = ele_name_club[0].text   # DF: Verein
    
    # Alle Boxen auf der Seite auslesen -> dritte Box sind die Zugänge, 4 Box sind die Abgänge
    
    ele_boxes = page_soup.find_all("div", {"class": "box"})
    
    ele_incomings = ele_boxes[2]
    ele_outgoings = ele_boxes[3]
    
    # ---------- BOX Zugänge -------------
    
    # Lese die Anzahl an Zugängen aus 
    
    ele_amount_of_incomings = ele_incomings.find_all("a", {"class": "spielprofil_tooltip"})
    amount_of_incomings = len(ele_amount_of_incomings)          # DF: Zugänge Anzahl
    
    # Finde den Footer der Box
    
    ele_footer_incomings = ele_incomings.find_all("tfoot")
    
    # Finde alle td  class="rechts" im Footer -> 
    # td1 = Summe der Transferkosten, td2 = Durchschnittsalter, td3 = Gesamtmarktwert |(der Zugänge)
    
    ele_footer_incomings_td = ele_footer_incomings[0].find_all("td", {"class": "rechts"})
    
    sum_cost_incomings = ele_footer_incomings_td[0].text         # DF: Zugänge Summe Transferkosten in Mio
    avg_age_incomings = ele_footer_incomings_td[1].text          # DF: Zugänge Durchschnittsalter
    sum_market_value_incomings = ele_footer_incomings_td[2].text # DF: Zugänge Summe Marktwert in Mio
    
    
    # ----------- Box Abgänge -------------
    
    # Lese die Anzahl an Abgängen aus 
    
    ele_amount_of_outgoings = ele_outgoings.find_all("a", {"class": "spielprofil_tooltip"})
    amount_of_outgoings = len(ele_amount_of_outgoings)           # DF: Abgänge Anzahl
    
    # Finde den Footer der Box
    
    ele_footer_outgoings = ele_outgoings.find_all("tfoot")
    
    # Finde alle td  class="rechts" im Footer -> 
    # td1 = Summe der Transferkosten, td2 = Durchschnittsalter, td3 = Gesamtmarktwert |(der Abgänge)
    
    ele_footer_outgoings_td = ele_footer_outgoings[0].find_all("td", {"class": "rechts"})
    
    sum_cost_outgoings = ele_footer_outgoings_td[0].text         # DF: Abgänge Summe Transferkosten in Mio
    avg_age_outgoings = ele_footer_outgoings_td[1].text          # DF: Abgänge Durchschnittsalter
    sum_market_value_outgoings = ele_footer_outgoings_td[2].text # DF: Abgänge Summe Marktwert in Mio
    
    # ------------ Datenbereinigung --------------
     
    # Verein von ""\nFC Bayern München\n" zu FC Bayern München
    
    name_club = name_club.replace('\n', '', 2)
    
    # Zugänge/Abgänge Transferkosten/Marktwert von "Summe: 116,50 Mio. €" zu 116,50 (in Mio)
    # Zugänge/Abgänge Alter von Durchschnittsalter der Zugänge: 26,6 zu 26,6
    
    sum_cost_incomings = string_to_number(sum_cost_incomings, 'costs')
    sum_market_value_incomings = string_to_number(sum_market_value_incomings, 'costs')
    
    sum_cost_outgoings = string_to_number(sum_cost_outgoings, 'costs')
    sum_market_value_outgoings = string_to_number(sum_market_value_outgoings, 'costs')
    
    avg_age_incomings = string_to_number(avg_age_incomings, 'age')
    avg_age_outgoings = string_to_number(avg_age_outgoings, 'age')
    
    
    
    
    
    df_season = pd.DataFrame({"Saison" : current_season, "Verein" : name_club,
                              "Zugänge Anzahl" : amount_of_incomings,
                              "Zugänge Summe Transferkosten in Mio." : sum_cost_incomings,
                              "Zugänge Durchschnittsalter" : avg_age_incomings,
                              "Zugänge Summe Marktwert in Mio." : sum_market_value_incomings,
                              "Abgänge Anzahl" : amount_of_outgoings,
                              "Abgänge Summe Transferkosten in Mio" : sum_cost_outgoings,
                              "Abgänge Durchschnittsalter" : avg_age_outgoings,
                              "Abgänge Summe Marktwert in Mio" : sum_market_value_outgoings}, index = [durchlauf])
    
    df_total = df_total.append(df_season)

    durchlauf += 1
    current_season += 1
    
    
    
    

In [12]:
df_total

,Saison,Verein,Zugänge Anzahl,Zugänge Summe Transferkosten in Mio.,Zugänge Durchschnittsalter,Zugänge Summe Marktwert in Mio.,Abgänge Anzahl,Abgänge Summe Transferkosten in Mio,Abgänge Durchschnittsalter,Abgänge Summe Marktwert in Mio
0,1960,FC Bayern München,14,-,"17,9",-,3,-,"26,3",-
1,1961,FC Bayern München,7,-,"19,6",-,7,-,"13,0",-
2,1962,FC Bayern München,6,-,"23,5",-,4,-,"24,0",-
3,1963,FC Bayern München,6,-,"14,2",-,6,-,"25,3",-
4,1964,FC Bayern München,4,-,"19,8",-,7,-,"23,0",-
5,1965,FC Bayern München,8,-,"17,1",-,3,-,"25,3",-
6,1966,FC Bayern München,7,-,"14,4",-,7,-,"13,1",-
7,1967,FC Bayern München,6,-,"21,5",-,8,-,"23,3",-
8,1968,FC Bayern München,6,-,"15,3",-,10,-,"20,9",-
9,1969,FC Bayern München,7,-,"20,7",-,4,-,"25,0",-


In [6]:
df_total.to_excel("Transferhistorie_ThunderBleilevens.xlsx")